<p style="float: left;"><a href="merge-sort.ipynb" target="_blank">Previous</a></p>
<p style="float: right;"><a href="for-comprehensions.ipynb" target="_blank">Next</a></p>
<p style="text-align:center;">Tour of Scala</p>
<div style="clear: both;"></div>

# Definition of class List II: Higher-Order Methods

- **Functional programming** languages **enable programmers to write general functions
which implement patterns by means of higher order functions**.

- We can identify several patterns of computation over lists, like:
    
    * Transforming every element of a list in some way.
      
    * Extracting from a list all elements satisfying a criterion.
      
    * Combine the elements of a list using some operator.

## Mapping

- A common operation is to transform each element of a list
and then return the lists of results.

    ```scala
    def map[B](f: A => B): List[B] = this match {
        case Nil     => this
        case x :: xs => f(x) :: xs map f
    }


    (1 to 3).toList map (_ * 2) // List(2, 4, 6)
    ```


## Foreach

- `foreach` applies a given function to all
elements of a list, but does not construct a list of results.

- The function is thus applied only for its side effect.

```scala
def foreach(f: A => Unit): Unit =
    this match {
        case Nil     => ()
        case x :: xs => {
            f(x)
            xs foreach f
        }


(1 to 3).toList foreach println // 1, 2, 3
```


## Filtering

- Another common operation is selecting from a list all elements that satisfies a `predicate`.

    ```scala
    def filter(p: A => Boolean): List[A] = this match {
        case Nil     => this
        case x :: xs => if p(x) then x :: xs filter p else xs filter p
    }

    (1 to 3).toList filter (_ >= 3) // List(3)
    
    ```

## Folding

- Another common operation is to combine the elements of a list with some operator.

    * `sum(List(x1, ..., xn))`  $ = 0 + x_1 + \dots + x_n$
      
    * `product(List(x1, ..., xn))` $ = 1 * x_1 * \dots * x_n$

### Fold left

```scala
def foldLeft[B](z: B)(op: (B, A) => B): B = this match {
    case Nil     => z                            // `foldLeft [1]`
    case x :: xs => (xs foldLeft op(z, x))(op)   // `foldLeft [2]`
}
```

- Applies a binary operator $\Theta$ to a start value $z$ and all elements of a list $x_1, x_2, .., x_n$, going left to right.

- **`foldLeft` generates the <span style="color:red">left associative expression</span> $ (\cdots ((z \ \Theta \ x_1) \ \Theta \ x_2) \ \cdots ) \ \Theta \ x_n$ simplified by the expression:** $z \ \Theta \ x_1 \ \Theta \ x_2 \ \cdots \ \Theta \ x_n$.

**Example:**

```scala
List(1, 2, 3).foldLeft(0)(_ + _)

```

- Step-by-step evaluation

    
    ```txt
    > List(1, 2, 3).foldLeft (0) (_ + _)        
    > List(2, 3).foldLeft (1) (_ + _)     // `foldLeft [2]`
    > List(3).foldLeft (3) (_ + _)        // `foldLeft [2]`
    > List().foldLeft (6) (_ + _)         // `foldLeft [2]`
    > 6                                   // `foldLeft [1]`
    ```

### Fold right


```scala
def foldRight[B](z: B)(op: (A, B) => B): B = this match {
    case Nil     => z                                // `foldRight [1]`
    case x :: xs => op(x, (xs foldRight z)(op))      // `foldRight [2]`
}
```

- Applies a binary operator $\Theta$ for all elements of a list $x_1, x_2, .., x_n$ and a start value $z$, going right to left.
  
- `foldRight` **produces a <span style="color:red">right-associative expression:</span>** $x_1 \ \Theta \ (x_2 \ \Theta \ \cdots \ (x_n \ \Theta \ z) \cdots )$.

**Example:**

```scala
List(1, 2, 3).foldRight(0)(_ + _)

```

Step-by-step evaluation:

```txt
> List(1, 2, 3).foldRight (0) (_ + _)        
> 1 + List(2, 3).foldRight 0 (_ + _)            // `foldRight [2]`
> 1 + (2 + List(3).foldRight 0) (_ + _))        // `foldRight [2]`
> 1 + (2 + (3 + List().foldRight 0) (_ + _)))   // `foldRight [2]`
> 1 + (2 + (3 + 0))                             // `foldRight [1]`
> 1 + (2 + 3)                                   // stack call
> 1 + 5                                         // stack call
> 6                                             // stack call
```

### `foldLeft` vs. `foldRight`

Both `foldLeft` and `foldRight` are very similar, but the key difference is the way the expressions are generated.

- `foldLeft` is _tail-recursive_ **(it evaluates the accumulator at each step)**
  
- `foldRight` is _non tail-cursive_ <span style="color:red">**(large lists can cause stack overflow)**</span>
  
- Use `foldRight` when you need to preserve the structure _right-to-left_ or you are working with lazzy data stractures such as `Strem` or `LazyList`.
    
    - You can _short-circuit_ without generating the whole structure.


#### Structure

- Variance of the structure using `foldLeft` and `foldRight`.

    ```scala
    import scala.language.postfixOps

    val yys = List (
        List.range(1, 5),
        List.range(6, 10),
        List.range(11, 15)
    )
    
    ```

    - Flatten list from _left-to-right_.
 
        ```scala
        (yys foldLeft (Nil)) ((xxs, xs) => xxs ::: xs) // List(1, 2, 3, 4, 6, 7, 8, 9, 11, 12, 13, 14)
        
        ```

        <br/>

    - Flatten list from _right-to-left_.
 
        ```scala
        (yys foldRight (Nil)) ((xs, xxs) => xxs ::: xs) // List(11, 12, 13, 14, 6, 7, 8, 9, 1, 2, 3, 4)

        ```


#### Short-circuit

- `foldRight` is can _short-circuit_ if:

    - you work with lazzy data structures,
    
    - and the operator itself **doesn’t force the second argument**, like in boolean `&&` or `||`.
      
    **Example:**


    ```scala
    def foldRight[B](z: B)(op: (A, B) => B): B = this match {
        case Nil     => z                                // `foldRight [1]`
        case x :: xs => op(x, (xs foldRight z)(op))      // `foldRight [2]`
    }
    
    ```

    <br/>
    
    ```scala
    (False :: True :: Nil).foldRight (True) (_ && _)
    
    ```

    <br/>
    
    - Step-by-step evaluation:
        
        ```txt
        > (False :: True :: Nil).foldRight (True) (_ && _)        
        > False && ((True :: Nil).foldRight (True) (_ + _))     // `foldRight [2]`
        > False                                                 // Short-cutting
        
        ```

    <br/>
    
- `foldLeft` can never _short-circuit_, you process the whole structure.

    **Example:**

    ```scala
    def foldLeft[B](z: B)(op: (B, A) => B): B = this match {
        case Nil     => z                            // `foldLeft [1]`
        case x :: xs => (xs foldLeft op(z, x))(op)   // `foldLeft [2]`
    }
    
    ```

    <br/>

    ```scala
    (False :: True :: Nil).foldLeft (True) (_ && _)
    
    ```

    <br/>

    - Step-by-step evaluation:
        
        ```scala
        >> (False :: True :: Nil).foldLeft (True) (_ && _)     
        >> (True :: Nil).foldLeft (False) (_ && _)       // `foldLeft [2]`
        >> (Nil).foldLeft (False) (_ && _)               // `foldLeft [2]`
        >> False                                         // `foldLeft [1]`
        
        ```

### ReduceLeft & ReduceRight

The method `reduceLeft` and `reduceRight` behaves similar to they analogous methods `foldLeft` and `foldRight`.
<span style="color:red">**But they require that the list is not empty.**</span>

```scala
def reduceLeft(op: (A, A) => A): A = this match {
    case Nil     => error("Nil.reduceLeft")    
    case x :: xs => (xs foldLeft x)(op)        
}

def reduceRight(op: (A, A) => A): A = this match {
    case Nil      => error("Nil.reduceRight")        
    case x :: Nil => x                              
    case x :: xs  => op(x, xs reduceRight op)        
}

```

## Nested Mappings

- You employ higher-order list processing functions to express many computations that are normally expressed as nested loops in imperative
languages.

- Consider the following problem: Given a positive integer $n$, find all
pairs of positive integers $i$ and $j$, where $1 \leq j \lt i < n$ such that $i + j$ is prime.

For instance, if $n = 7$ the pairs are:
    
| i | j | i + j |
|---|---|-------|
| 2 | 1 | 3     |
| 3 | 2 | 5     |
| 4 | 1 | 5     |
| 4 | 3 | 7     |
| 5 | 2 | 7     |
| 6 | 1 | 7     |
| 6 | 5 | 11    |



- Generates the sequence of all pairs $(i, j)$ of integers such that $1 \leq j < i < n$.

In [6]:
val n = 7

List.range(1, n)
    .map(i => List.range(1, i).map(j => (i, j)))

n: Int = 7
res6_1: List[List[(Int, Int)]] = List(
  List(),
  List((2, 1)),
  List((3, 1), (3, 2)),
  List((4, 1), (4, 2), (4, 3)),
  List((5, 1), (5, 2), (5, 3), (5, 4)),
  List((6, 1), (6, 2), (6, 3), (6, 4), (6, 5))
)

- Combine all sublists (flatting) using foldRight with the concat operator `:::`,

In [7]:
List.range(1, n)
    .map(i => List.range(1, i).map(j => (i, j)))
    .foldRight(List[(Int, Int)]()){(x, xs) => x ::: xs}

res7: List[(Int, Int)] = List(
  (2, 1),
  (3, 1),
  (3, 2),
  (4, 1),
  (4, 2),
  (4, 3),
  (5, 1),
  (5, 2),
  (5, 3),
  (5, 4),
  (6, 1),
  (6, 2),
  (6, 3),
  (6, 4),
  (6, 5)
)

- Filter the pairs that their sum is prime.

In [8]:
def isPrime(n: Int): Boolean = {
    def nestedIsPrime(n: Int, i: Int): Boolean = { 
        if (n <= 2) n == 2
        else if (n % i == 0) false
        else if (n < i * i) true
        else nestedIsPrime(n, i + 1)
    }
    nestedIsPrime(n, 2)
}

val n = 7

List.range(1, n)
    .map(i => List.range(1, i).map(j => (i, j)))
    .foldRight(List[(Int, Int)]()){(x, xs) => x ::: xs}
    .filter(pair => isPrime(pair._1 + pair._2))

defined function isPrime
n: Int = 7
res8_2: List[(Int, Int)] = List(
  (2, 1),
  (3, 2),
  (4, 1),
  (4, 3),
  (5, 2),
  (6, 1),
  (6, 5)
)

## Flattening Maps

- The combination of mapping and then concatenating sublists
resulting from the map is so common that there is a special method for it.

```scala
def flatMap[B](f: A => List[B]): List[B] = this match {
    case Nil     => Nil
    case x :: xs => f(x) ::: (xs flatMap f)
}

```

- Using `flatMap`, the pairs-whose-sum-is-prime could be expressed as:


    ```scala
    List.range(1, n)
        .flatMap(i => List.range(1, i).map(j => (i, j)))
        .filter(pair => isPrime(pair._1 + pair._2))

    ```

<p style="float: left;"><a href="merge-sort.ipynb" target="_blank">Previous</a></p>
<p style="float: right;"><a href="for-comprehensions.ipynb" target="_blank">Next</a></p>
<p style="text-align:center;">Tour of Scala</p>
<div style="clear: both;"></div>